Importing Libraries

In [1]:
import pandas as pd
import random
from numpy.random import choice
from pyspark.mllib.clustering import KMeans, KMeansModel
from numpy import array

Loading data from csv

In [2]:
df = pd.read_csv("2016_ipl_data.csv")


strike = list(df["Strike"])
bowler = list(df["Bowler"])
#bowler = list(set(bowler))
non_strike = list(df["Non_Strike"])
batsmen = list(set(strike+non_strike))
batsmen.sort()

#combinations in the form of (stike_batsman,bowler)
combinations = list(set(zip(strike,bowler)))
combinations.sort()

comb_stat = dict()

In [3]:
#stats = [outs,0,1,2,3,4,6,number_of_balls_faced]

for i in combinations:
    comb_stat[i] = [0,0,0,0,0,0,0,0]

#initializing stats of each combination ---- Player vs Player statistics
for index in range(len(df["Out"])):
    comb_stat[(df.loc[index]["Strike"],df.loc[index]["Bowler"])][7] = 1 + comb_stat[(df.loc[index]["Strike"],df.loc[index]["Bowler"])][7]
    
    
    if(df.loc[index]["Out"]!='""' and df.loc[index]["Out"]!='run out'):
        comb_stat[(df.loc[index]["Strike"],df.loc[index]["Bowler"])][0] = 1+comb_stat[(df.loc[index]["Strike"],df.loc[index]["Bowler"])][0]
    elif(df.loc[index]["Runs"]==0):
        comb_stat[(df.loc[index]["Strike"],df.loc[index]["Bowler"])][1] = 1+comb_stat[(df.loc[index]["Strike"],df.loc[index]["Bowler"])][1]
    elif(df.loc[index]["Runs"]==1):
        comb_stat[(df.loc[index]["Strike"],df.loc[index]["Bowler"])][2] = 1+comb_stat[(df.loc[index]["Strike"],df.loc[index]["Bowler"])][2]
    elif(df.loc[index]["Runs"]==2):
        comb_stat[(df.loc[index]["Strike"],df.loc[index]["Bowler"])][3] = 1+comb_stat[(df.loc[index]["Strike"],df.loc[index]["Bowler"])][3]
    elif(df.loc[index]["Runs"]==3):
        comb_stat[(df.loc[index]["Strike"],df.loc[index]["Bowler"])][4] = 1+comb_stat[(df.loc[index]["Strike"],df.loc[index]["Bowler"])][4]
    elif(df.loc[index]["Runs"]==4):
        comb_stat[(df.loc[index]["Strike"],df.loc[index]["Bowler"])][5] = 1+comb_stat[(df.loc[index]["Strike"],df.loc[index]["Bowler"])][5]
    elif(df.loc[index]["Runs"]==6):
        comb_stat[(df.loc[index]["Strike"],df.loc[index]["Bowler"])][6] = 1+comb_stat[(df.loc[index]["Strike"],df.loc[index]["Bowler"])][6]

In [4]:
#weighted random number   
#from numpy.random import choice
#draw = choice(list_of_candidates, number_of_items_to_pick, p=probability_distribution)
    
#Generating cumulative probabilities for combinations -- helpful for predicting runs
for i in combinations:
    l = comb_stat[i][1:]
    probs = [round(i/l[-1],2) for i in l[:-1]]
    cum_prob = [probs[0]]
    for j in range(1,len(probs)):
        cum_prob.append(cum_prob[j-1]+probs[j])
    
    comb_stat[i].append(cum_prob)

#stats = [outs,0,1,2,3,4,6,number_of_balls_faced]   
print(comb_stat[i])

[0, 4, 0, 0, 0, 0, 0, 4, [1.0, 1.0, 1.0, 1.0, 1.0, 1.0]]


In [5]:
#random number between 0 and 1      
#float("{0:.2f}".format(random.random()))

#one method to predict runs using cumulative probability
def randindex(k,cum_prob):
    #print(k,cum_prob)
    '''
    #if combination doesn't exist do group vs group instead of this.
    if(cum_prob == [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]):
        return [0,1,2,3,4,6][random.randint(0,5)]
    '''
    if(k<=cum_prob[0]):
        return 0
    elif(k<=cum_prob[1]):
        return 1
    elif(k<=cum_prob[2]):
        return 2
    elif(k<=cum_prob[3]):
        return 3
    elif(k<=cum_prob[4]):
        return 4
    else:
        return 6

    
#returns predicted runs by inputting a random number between (0,1) and cumulative probability
def get_run(k,lst):
    ind = []
    repind=[]
    s = 0
    for i in range(len(lst)):
        if(k > lst[i]):
            ind.append(i)
        else:
            break
    
    if(len(ind)==0):
        index=0
    else:
        index = ind[len(ind)-1] + 1
    if(index==len(lst)):
        return 6
    else:
        repind.append(index)
        for i in range(index,len(lst)):
            if(lst[index]==lst[i]):
                repind.append(i)
            else:
                break
       
        k = random.randint(repind[0],repind[-1])
        if(k==5):
            return 3
        else:
            return k

def get_run_old(k,cum_prob):
    prob = [cum_prob[0]]
    for i in range(1,len(cum_prob)):
        temp = cum_prob[i] - cum_prob[i-1]
        prob.append(round(temp,2))
    runs = choice([0,1,2,3,4,6],1, p=prob)
    return runs
        
        
        
#loading batsmen cluster model -- helpful for determining confidence of batsman
bat_sameModel = KMeansModel.load(sc, "hdfs://localhost:54310/project/bat_result")
bat_parsedData = sc.textFile("hdfs://localhost:54310/project/batsmen_stat.csv") \
    .map(lambda line: line.split(",")) \
    .filter(lambda line: len(line)>1 and line[1]!="Name") \
    .map(lambda line: array([float(line[2]),float(line[3]),float(line[4]),float(line[5]),\
                             float(line[6]),float(line[7]),float(line[8]),float(line[9]),float(line[10])]))
bat_names = sc.textFile("hdfs://localhost:54310/project/batsmen_stat.csv")\
.map(lambda line: line.split(",")).filter(lambda line: len(line)>1 and line[1]!="Name").map(lambda line: array([line[1]]))
bat_names = bat_names.map(lambda x:x[0])
bat_player_class = bat_parsedData.map(lambda x: (x,bat_sameModel.predict(x)))
bat_name_class = bat_names.zip(bat_player_class)
bat_classes = bat_name_class.collect()

#loading bowlers cluster model -- helpful for determining strength of bowler
bowl_sameModel = KMeansModel.load(sc, "hdfs://localhost:54310/project/bowl_result")
bowl_parsedData = sc.textFile("hdfs://localhost:54310/project/bowler_stat.csv") \
    .map(lambda line: line.split(",")) \
    .filter(lambda line: len(line)>1 and line[1]!="Name") \
    .map(lambda line: array([float(line[2]),float(line[3]),float(line[4]),float(line[5]),\
                             float(line[6]),float(line[7]),float(line[8])]))

bowl_names = sc.textFile("hdfs://localhost:54310/project/bowler_stat.csv") \
    .map(lambda line: line.split(",")).filter(lambda line: len(line)>1 and line[1]!="Name").map(lambda line: array([line[1]]))
bowl_names = bowl_names.map(lambda x:x[0])
bowl_player_class = bowl_parsedData.map(lambda x: (x,bowl_sameModel.predict(x)))
bowl_name_class = bowl_names.zip(bowl_player_class)
bowl_classes = bowl_name_class.collect()


#returns class of a batsman , based on name
def class_by_name_bat(name):
    for i in bat_classes:
        if(i[0]==name):
            return i[1][1]
        
    return -1
    
#returns class of bowler based on name
def class_by_name_bowl(name):
    for i in bowl_classes:
        if(i[0]==name):
            return i[1][1]
        
    return -1


#computes group vs group statistics incase of a new combination
def grp_vs_grp(batname,bowlname):
    batclass = class_by_name_bat(batname)
    bowlclass = class_by_name_bowl(bowlname)
    strike = list(df["Strike"])
    bowler = list(df["Bowler"])
    #bowler = list(set(bowler))
    non_strike = list(df["Non_Strike"])
    batsmen = list(set(strike+non_strike))
    batsmen.sort()
    #new combination out of existing batsmen and bowlers
    if(batclass!=-1 and bowlclass!=-1):
        #print("Sup")
        a = []
        for i in comb_stat.keys():
            if(class_by_name_bat(i[0])==batclass and class_by_name_bowl(i[1])==bowlclass):
                a.append(comb_stat[i][:-1])
        l = list(zip(*a))
        x = []
        for i in l:
            x.append(sum(i)/len(a))
        l = x[1:]
        probs = [round(i/l[-1],2) for i in l[:-1]]
        cum_prob = [probs[0]]
        for j in range(1,len(probs)):
            cum_prob.append(cum_prob[j-1]+probs[j])
        return cum_prob
    #batsman debut , old bowler
    elif(batclass==-1 and bowlclass!=-1):
        a =[]
        for i in comb_stat.keys():
            if(class_by_name_bowl(i[1])==bowlclass):
                a.append(comb_stat[i][:-1])
        l = list(zip(*a))
        x = []
        for i in l:
            x.append(sum(i)/len(a))
        l = x[1:]
        probs = [round(i/l[-1],2) for i in l[:-1]]
        cum_prob = [probs[0]]
        for j in range(1,len(probs)):
            cum_prob.append(cum_prob[j-1]+probs[j])
        return cum_prob
    #old batsman , bowler debut
    elif(batclass!=-1 and bowlclass==-1):
        a = []
        for i in comb_stat.keys():
            if(class_by_name_bat(i[0])==batclass):
                a.append(comb_stat[i][:-1])
        l = list(zip(*a))
        x = []
        for i in l:
            x.append(sum(i)/len(a))
        l = x[1:]
        probs = [round(i/l[-1],2) for i in l[:-1]]
        cum_prob = [probs[0]]
        for j in range(1,len(probs)):
            cum_prob.append(cum_prob[j-1]+probs[j])
        return cum_prob
    #both batsman and bowler debut
    elif(batclass==-1 and bowlclass==-1):
        a = []
        for i in comb_stat.keys():
            a.append(comb_stat[i][:-1])
        l = list(zip(*a))
        x = []
        for i in l:
            x.append(sum(i)/len(a))
        l = x[1:]
        probs = [round(i/l[-1],2) for i in l[:-1]]
        cum_prob = [probs[0]]
        for j in range(1,len(probs)):
            cum_prob.append(cum_prob[j-1]+probs[j])
        return cum_prob
        

#Adding hierarchy to batsmen based on the cluster they belong to ..... Based on avg runs scored by the clusters
avg_runs_per_class = []
for i in range(10):
    avg_runs_per_class.append((bat_sameModel.centers[i][1],i))

#print(sameModel.centers)
avg_runs_per_class.sort()
sorted_run_class = [i[1] for i in avg_runs_per_class]
print(sorted_run_class)

avg_wickets_per_class = []
for i in range(5):
    avg_wickets_per_class.append((bowl_sameModel.centers[i][2],i))

avg_wickets_per_class.sort()
sorted_wickets_class = [i[1] for i in avg_wickets_per_class]
print(sorted_wickets_class)

#just to check how a batsman performs against a bowler for a certain number of bowls
def simulate_combination(batsman,bowler,balls):
    cum_prob = comb_stat[(batsman,bowler)][-1]
    print(cum_prob)
    bat_class = class_by_name_bat(batsman)
    confidence = sorted_run_class.index(bat_class)*10
    if(confidence<20):
        confidence = 20
    #print(confidence)
    l = []
    for i in range(balls):
        k = float("{0:.2f}".format(random.random()))
        a = get_run(k,cum_prob)
        
        if(a==0):
            confidence = confidence-2
        else:
            confidence = confidence+a
            if(confidence>=100):
                confidence = 100
        prob = confidence/100
        #print(prob)
        draw = choice([0,1],1, p=[prob,1-prob])
        if(draw==0):
            l.append(a)
        else:
            l.append('W')
            break
    #print(l)
    return l


#simulates a combination of a batsman facing one ball of a bowler with certain confidence
#confidence is dependent on the type of cluster the batsman belongs to 
def sim_comb(batsman,bowler,balls,confidence):
    #if(confidence>=100):
    #    confidence = 100
    if((batsman,bowler) in comb_stat.keys()):
        cum_prob = comb_stat[(batsman,bowler)][-1]
    #print(cum_prob)
    else:
        k = float("{0:.2f}".format(random.random()))
        cum_prob = grp_vs_grp(batsman,bowler)
        #print(k,cum_prob,batsman,bowler)
        a = get_run(k,cum_prob)
        
        if(a==0):
            confidence = confidence-2
        else:
            confidence = confidence+a
            if(confidence>=100):
                confidence = 100
        prob = confidence/100
        #print(prob)
        bowler_class = class_by_name_bowl(bowler)
        if(bowler_class==-1):
            bowler_conf=50
        else:
            bowler_conf = sorted_wickets_class.index(bowler_class)*20
        if(bowler_conf<50):
            bowler_conf=50
        bowler_prob = bowler_conf/100
        #good_ball = choice([0,1],1, p=[1-bowler_prob,bowler_prob])
        good_ball = choice([0,1],1, p=[0.9,0.1])
        if(good_ball==1):
            #draw = choice([0,1],1, p=[prob,1-prob])
            draw = choice([0,1],1, p=[0.8,.2])
            if(draw==0):
                return a,confidence
            else:
                return 'W',confidence
        else:
            return a,confidence
    l = []
    for i in range(balls):
        k = float("{0:.2f}".format(random.random()))
        a = get_run(k,cum_prob)
        #print("fine",k,cum_prob)
        if(a==0):
            confidence = confidence-2
        else:
            confidence = confidence+a
            if(confidence>=100):
                confidence = 100
        prob = confidence/100
        #print(prob)
        bowler_class = class_by_name_bowl(bowler)
        bowler_conf = sorted_wickets_class.index(bowler_class)*20
        if(bowler_conf<50):
            bowler_conf=50
        bowler_prob = bowler_conf/100
        #good_ball = choice([0,1],1, p=[1-bowler_prob,bowler_prob])
        good_ball = choice([0,1],1, p=[0.8,0.2])
        if(good_ball==1):
            #draw = choice([0,1],1, p=[prob,1-prob])
            draw = choice([0,1],1, p=[0.8,0.2])
            if(draw==0):
                return a,confidence
            else:
                return 'W',confidence
        else:
            return a,confidence
    #print(l)
    return l[0],confidence

#simulate first innings of a match .....
def sim_match(bat_lineup,bowling_order):
    wickets = 0
    strike = bat_lineup[0]
    non_strike = bat_lineup[1]
    balls = 120
    bat_class = class_by_name_bat(strike)
    if(bat_class==-1):
        strike_conf = 50
    else:
        strike_conf = sorted_run_class.index(bat_class)*10
    #strike_conf = sorted_run_class.index(class_by_name_bat(strike))*10
    if(strike_conf<20):
        strike_conf = 20
    bat_class = class_by_name_bat(non_strike)
    if(bat_class==-1):
        non_strike_conf = 50
    else:
        non_strike_conf = sorted_run_class.index(bat_class)*10
    #non_strike_conf = sorted_run_class.index(class_by_name_bat(non_strike))*10
    if(non_strike_conf<20):
        non_strike_conf=20
    bat_stat = dict()
    bowl_stat = dict()
    # [runs,dots,4s,6s,balls,strike rate , out_by]
    for i in bat_lineup:
        bat_stat[i] = [0,0,0,0,0,0, ""]
    #bat_stat[strike] = [0,0,0,0,0,0, ""] 
    #bat_stat[non_strike] = [0,0,0,0,0,0,""]
    bowlers = list(set(bowling_order))
    for i in bowlers:
        # [balls , runs , wickets , economy]
        bowl_stat[i] = [0,0,0,0]
    j = 0
    b_index = 2
    while(balls>=0 and wickets!=10):
        if(j==20):
            break
        for i in range(6):
            if(balls<0):
                break
            k,strike_conf = sim_comb(strike,bowling_order[j],1,strike_conf)
            #update batsman and bowler stat
            print(k,strike,non_strike,bowling_order[j],balls)
            if(k=='W'):
                bat_stat[strike][1] = bat_stat[strike][1] + 1
                bat_stat[strike][4] = bat_stat[strike][4] + 1
                bat_stat[strike][5] = bat_stat[strike][0]/bat_stat[strike][4]*100
                bat_stat[strike][-1] = bowling_order[j]
                wickets = wickets+1
                bowl_stat[bowling_order[j]][2] = 1 + bowl_stat[bowling_order[j]][2]
                bowl_stat[bowling_order[j]][0] = 1 + bowl_stat[bowling_order[j]][0]
                bowl_stat[bowling_order[j]][3] = bowl_stat[bowling_order[j]][1]/(bowl_stat[bowling_order[j]][0]/6)
                if(wickets==10):
                    break
                strike = bat_lineup[b_index]
                bat_class = class_by_name_bat(strike)
                if(bat_class==-1):
                    strike_conf = 50
                else:
                    strike_conf = sorted_run_class.index(bat_class)*10
                if(strike_conf<20):
                    strike_conf = 20
                
                
                b_index = 1+b_index
                #print(b_index)
                
            else:
                bat_stat[strike][0] = bat_stat[strike][0] + k
                if(k==0):
                    bat_stat[strike][1] = bat_stat[strike][1] + 1
                elif(k==4):
                    bat_stat[strike][2] = bat_stat[strike][2] + 1
                elif(k==6):
                    bat_stat[strike][3] = bat_stat[strike][3] + 1
                bat_stat[strike][4] = bat_stat[strike][4] + 1
                bat_stat[strike][5] = bat_stat[strike][0]/bat_stat[strike][4]*100
                #strike_conf = strike_conf+k
                bowl_stat[bowling_order[j]][0] = 1 + bowl_stat[bowling_order[j]][0]
                bowl_stat[bowling_order[j]][1] = k + bowl_stat[bowling_order[j]][1]
                bowl_stat[bowling_order[j]][3] = bowl_stat[bowling_order[j]][1]/(bowl_stat[bowling_order[j]][0]/6)
            
            if(k!='W' and k%2!=0):
                #strike_conf = strike_conf+k
                strike,non_strike = non_strike,strike
                strike_conf,non_strike_conf = non_strike_conf,strike_conf
            balls = balls -1
            
            
        strike,non_strike = non_strike,strike
        strike_conf,non_strike_conf = non_strike_conf,strike_conf
        j = j+1
    runs = sum([bat_stat[i][0] for i in bat_stat.keys()])
    #for i in bat_stat.keys():
    #    print(i)
    print("\nFor batsman : [runs,dots,4s,6s,balls,strike rate , out_by]\n")
    print(bat_stat)
    print("\nFor bowler : [balls , runs , wickets , economy]\n")
    print(bowl_stat)
    print(runs)
    print(wickets)
    return runs,wickets


#simulating 2nd innings given target
def sim_chase(bat_lineup,bowling_order,chase_target):
    wickets = 0
    strike = bat_lineup[0]
    non_strike = bat_lineup[1]
    balls = 120
    bat_class = class_by_name_bat(strike)
    if(bat_class==-1):
        strike_conf = 50
    else:
        strike_conf = sorted_run_class.index(bat_class)*10
    #strike_conf = sorted_run_class.index(class_by_name_bat(strike))*10
    if(strike_conf<20):
        strike_conf = 20
    bat_class = class_by_name_bat(non_strike)
    if(bat_class==-1):
        non_strike_conf = 50
    else:
        non_strike_conf = sorted_run_class.index(bat_class)*10
    #non_strike_conf = sorted_run_class.index(class_by_name_bat(non_strike))*10
    if(non_strike_conf<20):
        non_strike_conf=20
    bat_stat = dict()
    bowl_stat = dict()
    # [runs,dots,4s,6s,balls,strike rate , out_by]
    for i in bat_lineup:
        bat_stat[i] = [0,0,0,0,0,0, ""]
    #bat_stat[strike] = [0,0,0,0,0,0, ""] 
    #bat_stat[non_strike] = [0,0,0,0,0,0,""]
    bowlers = list(set(bowling_order))
    for i in bowlers:
        # [balls , runs , wickets , economy]
        bowl_stat[i] = [0,0,0,0]
    j = 0
    b_index = 2
    total_runs = 0
    total_wickets = 0
    while(balls>=0 and wickets!=10):
        if(j==20):
            break
        for i in range(6):
            if(balls<0):
                break
            if(total_runs>chase_target):
                print("Chasing Team wins")
                balls = -1
                break

            k,strike_conf = sim_comb(strike,bowling_order[j],1,strike_conf)
            #update batsman and bowler stat
            print(k,strike,non_strike,bowling_order[j],balls)
            if(k=='W'):
                total_wickets = 1 + total_wickets
                bat_stat[strike][1] = bat_stat[strike][1] + 1
                bat_stat[strike][4] = bat_stat[strike][4] + 1
                bat_stat[strike][5] = bat_stat[strike][0]/bat_stat[strike][4]*100
                bat_stat[strike][-1] = bowling_order[j]
                wickets = wickets+1
                bowl_stat[bowling_order[j]][2] = 1 + bowl_stat[bowling_order[j]][2]
                bowl_stat[bowling_order[j]][0] = 1 + bowl_stat[bowling_order[j]][0]
                bowl_stat[bowling_order[j]][3] = bowl_stat[bowling_order[j]][1]/(bowl_stat[bowling_order[j]][0]/6)
                if(wickets==10):
                    break
                strike = bat_lineup[b_index]
                bat_class = class_by_name_bat(strike)
                if(bat_class==-1):
                    strike_conf = 50
                else:
                    strike_conf = sorted_run_class.index(bat_class)*10
                if(strike_conf<20):
                    strike_conf = 20
                
                
                b_index = 1+b_index
                #print(b_index)
                
            else:
                total_runs = k + total_runs
                bat_stat[strike][0] = bat_stat[strike][0] + k
                if(k==0):
                    bat_stat[strike][1] = bat_stat[strike][1] + 1
                elif(k==4):
                    bat_stat[strike][2] = bat_stat[strike][2] + 1
                elif(k==6):
                    bat_stat[strike][3] = bat_stat[strike][3] + 1
                bat_stat[strike][4] = bat_stat[strike][4] + 1
                bat_stat[strike][5] = bat_stat[strike][0]/bat_stat[strike][4]*100
                #strike_conf = strike_conf+k
                bowl_stat[bowling_order[j]][0] = 1 + bowl_stat[bowling_order[j]][0]
                bowl_stat[bowling_order[j]][1] = k + bowl_stat[bowling_order[j]][1]
                bowl_stat[bowling_order[j]][3] = bowl_stat[bowling_order[j]][1]/(bowl_stat[bowling_order[j]][0]/6)
            
            if(k!='W' and k%2!=0):
                #strike_conf = strike_conf+k
                strike,non_strike = non_strike,strike
                strike_conf,non_strike_conf = non_strike_conf,strike_conf
            balls = balls -1
            
            
        strike,non_strike = non_strike,strike
        strike_conf,non_strike_conf = non_strike_conf,strike_conf
        j = j+1
    runs = sum([bat_stat[i][0] for i in bat_stat.keys()])
    #for i in bat_stat.keys():
    #    print(i)
    if(runs<chase_target):
        print("Bowling Team Wins")
    print("\nFor batsman : [runs,dots,4s,6s,balls,strike rate , out_by]\n")
    print(bat_stat)
    print("\nFor bowler : [balls , runs , wickets , economy]\n")
    print(bowl_stat)
    print(runs)
    print(wickets)
    return runs,wickets



#simulates a complete match
def match(innings1 , innings2):

    target,wickets = sim_match(innings1[0],innings1[1])

    chase_score,wickets1 = sim_chase(innings2[0],innings2[1],target)

    print("Match Statistics :")
    print("1st Innings :")
    print("Score : "+str(target)+"\t Wickets : "+str(wickets))
    print("2nd Innings :")
    print("Score : "+str(chase_score)+"\t Wickets : "+str(wickets1))
    







[8, 1, 7, 9, 6, 2, 3, 4, 0, 5]
[0, 2, 3, 4, 1]


In [6]:
class_by_name_bat('V Kohli')

5

In [31]:
sim = simulate_combination("V Kohli","A Mishra",1)
print(sim)

[0.19, 0.81, 0.81, 0.81, 0.93, 0.99]
[1]


In [8]:
import random

#1st innings
bat_lineup = ["LMP Simmons","RG Sharma","HH Pandya","JC Buttler","AT Rayudu","KA Pollard","S Gopal","Harbhajan Singh","R Vinay Kumar","MJ McClenaghan","JJ Bumrah"]
print(len(bat_lineup))
bowling_order = ["RP Singh","I Sharma","RP Singh","I Sharma","MR Marsh","I Sharma","MR Marsh","R Bhatia","MR Marsh","R Bhatia","M Ashwin","R Bhatia","M Ashwin","R Bhatia","M Ashwin","R Ashwin","M Ashwin","MR Marsh","RP Singh","I Sharma"]
print(len(bowling_order))


# [runs,dots,4s,6s,balls,strike rate , out_by]
sim_match(bat_lineup,bowling_order)





11
20
W LMP Simmons RG Sharma RP Singh 120
W HH Pandya RG Sharma RP Singh 119
1 JC Buttler RG Sharma RP Singh 118
1 RG Sharma JC Buttler RP Singh 117
W JC Buttler RG Sharma RP Singh 116
0 AT Rayudu RG Sharma RP Singh 115
6 RG Sharma AT Rayudu I Sharma 114
6 RG Sharma AT Rayudu I Sharma 113
6 RG Sharma AT Rayudu I Sharma 112
6 RG Sharma AT Rayudu I Sharma 111
6 RG Sharma AT Rayudu I Sharma 110
6 RG Sharma AT Rayudu I Sharma 109
1 AT Rayudu RG Sharma RP Singh 108
0 RG Sharma AT Rayudu RP Singh 107
3 RG Sharma AT Rayudu RP Singh 106
0 AT Rayudu RG Sharma RP Singh 105
0 AT Rayudu RG Sharma RP Singh 104
4 AT Rayudu RG Sharma RP Singh 103
6 RG Sharma AT Rayudu I Sharma 102
6 RG Sharma AT Rayudu I Sharma 101
6 RG Sharma AT Rayudu I Sharma 100
6 RG Sharma AT Rayudu I Sharma 99
2 RG Sharma AT Rayudu I Sharma 98
6 RG Sharma AT Rayudu I Sharma 97
W AT Rayudu RG Sharma MR Marsh 96
0 KA Pollard RG Sharma MR Marsh 95
4 KA Pollard RG Sharma MR Marsh 94
0 KA Pollard RG Sharma MR Marsh 93
W KA Pollard 

(122, 10)

In [12]:
#2nd innings
bat_lineup = ["AM Rahane","F du Plessis","KP Pietersen","SPD Smith","MR Marsh","MS Dhoni","R Bhatia","R Ashwin","M Ashwin","I Sharma","RP Singh"]
print(len(bat_lineup))
bowling_order = ["MJ McClenaghan","JJ Bumrah","MJ McClenaghan","JJ Bumrah","MJ McClenaghan","JJ Bumrah","R Vinay Kumar","S Gopal","R Vinay Kumar","Harbhajan Singh","S Gopal","Harbhajan Singh","S Gopal","Harbhajan Singh","HH Pandya","R Vinay Kumar","JJ Bumrah","MJ McClenaghan","Harbhajan Singh","R Vinay Kumar"]

sim_chase(bat_lineup,bowling_order,54)

11
6 AM Rahane F du Plessis MJ McClenaghan 120
0 AM Rahane F du Plessis MJ McClenaghan 119
0 AM Rahane F du Plessis MJ McClenaghan 118
0 AM Rahane F du Plessis MJ McClenaghan 117
0 AM Rahane F du Plessis MJ McClenaghan 116
0 AM Rahane F du Plessis MJ McClenaghan 115
W F du Plessis AM Rahane JJ Bumrah 114
1 KP Pietersen AM Rahane JJ Bumrah 113
0 AM Rahane KP Pietersen JJ Bumrah 112
3 AM Rahane KP Pietersen JJ Bumrah 111
W KP Pietersen AM Rahane JJ Bumrah 110
6 SPD Smith AM Rahane JJ Bumrah 109
0 AM Rahane SPD Smith MJ McClenaghan 108
W AM Rahane SPD Smith MJ McClenaghan 107
1 MR Marsh SPD Smith MJ McClenaghan 106
1 SPD Smith MR Marsh MJ McClenaghan 105
0 MR Marsh SPD Smith MJ McClenaghan 104
W MR Marsh SPD Smith MJ McClenaghan 103
6 SPD Smith MS Dhoni JJ Bumrah 102
6 SPD Smith MS Dhoni JJ Bumrah 101
6 SPD Smith MS Dhoni JJ Bumrah 100
6 SPD Smith MS Dhoni JJ Bumrah 99
6 SPD Smith MS Dhoni JJ Bumrah 98
6 SPD Smith MS Dhoni JJ Bumrah 97
0 MS Dhoni SPD Smith MJ McClenaghan 96
4 MS Dhoni SPD

(58, 4)

In [20]:
innings1 = [["LMP Simmons","RG Sharma","HH Pandya","JC Buttler","AT Rayudu","KA Pollard","S Gopal","Harbhajan Singh","R Vinay Kumar","MJ McClenaghan","JJ Bumrah"],["RP Singh","I Sharma","RP Singh","I Sharma","MR Marsh","I Sharma","MR Marsh","R Bhatia","MR Marsh","R Bhatia","M Ashwin","R Bhatia","M Ashwin","R Bhatia","M Ashwin","R Ashwin","M Ashwin","MR Marsh","RP Singh","I Sharma"]]
innings2 = [["AM Rahane","F du Plessis","KP Pietersen","SPD Smith","MR Marsh","MS Dhoni","R Bhatia","R Ashwin","M Ashwin","I Sharma","RP Singh"],["MJ McClenaghan","JJ Bumrah","MJ McClenaghan","JJ Bumrah","MJ McClenaghan","JJ Bumrah","R Vinay Kumar","S Gopal","R Vinay Kumar","Harbhajan Singh","S Gopal","Harbhajan Singh","S Gopal","Harbhajan Singh","HH Pandya","R Vinay Kumar","JJ Bumrah","MJ McClenaghan","Harbhajan Singh","R Vinay Kumar"]]

match(innings1 , innings2)


0 LMP Simmons RG Sharma RP Singh 120
0 LMP Simmons RG Sharma RP Singh 119
0 LMP Simmons RG Sharma RP Singh 118
0 LMP Simmons RG Sharma RP Singh 117
3 LMP Simmons RG Sharma RP Singh 116
0 RG Sharma LMP Simmons RP Singh 115
W LMP Simmons RG Sharma I Sharma 114
W HH Pandya RG Sharma I Sharma 113
2 JC Buttler RG Sharma I Sharma 112
2 JC Buttler RG Sharma I Sharma 111
2 JC Buttler RG Sharma I Sharma 110
2 JC Buttler RG Sharma I Sharma 109
0 RG Sharma JC Buttler RP Singh 108
3 RG Sharma JC Buttler RP Singh 107
1 JC Buttler RG Sharma RP Singh 106
1 RG Sharma JC Buttler RP Singh 105
1 JC Buttler RG Sharma RP Singh 104
4 RG Sharma JC Buttler RP Singh 103
1 JC Buttler RG Sharma I Sharma 102
6 RG Sharma JC Buttler I Sharma 101
6 RG Sharma JC Buttler I Sharma 100
6 RG Sharma JC Buttler I Sharma 99
6 RG Sharma JC Buttler I Sharma 98
6 RG Sharma JC Buttler I Sharma 97
6 JC Buttler RG Sharma MR Marsh 96
6 JC Buttler RG Sharma MR Marsh 95
W JC Buttler RG Sharma MR Marsh 94
W AT Rayudu RG Sharma MR Mar

In [86]:
#ipl 2018 final
innings1 = [["SP Goswami","S Dhawan","KS Williamson","Shakib Al Hasan","YK Pathan","DJ Hooda","CR Brathwaite","Rashid Khan","B Kumar","S Kaul","Sandeep Sharma"],["DL Chahar","L Ngidi","DL Chahar","L Ngidi","DL Chahar","SN Thakur","KV Sharma","DJ Bravo","RA Jadeja","DL Chahar","RA Jadeja","DJ Bravo","KV Sharma","SN Thakur","KV Sharma","DJ Bravo","L Ngidi","DJ Bravo","L Ngidi","SN Thakur"]]
innings2 = [["SR Watson","F du Plessis","SK Raina","AT Rayudu","MS Dhoni","DJ Bravo","RA Jadeja","DL Chahar","KV Sharma","SN Thakur","L Ngidi"],["B Kumar","Sandeep Sharma","B Kumar","Sandeep Sharma","B Kumar","Sandeep Sharma","S Kaul","Rashid Khan","S Kaul","Rashid Khan","Shakib Al Hasan","CR Brathwaite","Sandeep Sharma","CR Brathwaite","Rashid Khan","B Kumar","Rashid Khan","S Kaul","CR Brathwaite","S Kaul"]]

match(innings1 , innings2)


1 SP Goswami S Dhawan DL Chahar 120
0 S Dhawan SP Goswami DL Chahar 119
0 S Dhawan SP Goswami DL Chahar 118
2 S Dhawan SP Goswami DL Chahar 117
0 S Dhawan SP Goswami DL Chahar 116
1 S Dhawan SP Goswami DL Chahar 115
6 S Dhawan SP Goswami L Ngidi 114
0 S Dhawan SP Goswami L Ngidi 113
0 S Dhawan SP Goswami L Ngidi 112
6 S Dhawan SP Goswami L Ngidi 111
2 S Dhawan SP Goswami L Ngidi 110
6 S Dhawan SP Goswami L Ngidi 109
0 SP Goswami S Dhawan DL Chahar 108
1 SP Goswami S Dhawan DL Chahar 107
0 S Dhawan SP Goswami DL Chahar 106
1 S Dhawan SP Goswami DL Chahar 105
1 SP Goswami S Dhawan DL Chahar 104
0 S Dhawan SP Goswami DL Chahar 103
1 SP Goswami S Dhawan L Ngidi 102
2 S Dhawan SP Goswami L Ngidi 101
0 S Dhawan SP Goswami L Ngidi 100
0 S Dhawan SP Goswami L Ngidi 99
1 S Dhawan SP Goswami L Ngidi 98
2 SP Goswami S Dhawan L Ngidi 97
1 S Dhawan SP Goswami DL Chahar 96
1 SP Goswami S Dhawan DL Chahar 95
0 S Dhawan SP Goswami DL Chahar 94
1 S Dhawan SP Goswami DL Chahar 93
4 SP Goswami S Dhawan D

In [74]:
#10 matches

#match 1
innings1 = [["MA Agarwal","Q de Kock","SS Iyer","KK Nair","SV Samson","P Negi","CR Brathwaite","CH Morris","NM Coulter-Nile","A Mishra","Z Khan"],["AD Russell","UT Yadav","AD Russell","JW Hastings","AD Russell","JW Hastings","UT Yadav","C Munro","UT Yadav","GB Hogg","PP Chawla","GB Hogg","PP Chawla","GB Hogg","PP Chawla","GB Hogg","PP Chawla","JW Hastings","AD Russell","UT Yadav"]]
innings2 = [["RV Uthappa","G Gambhir","MK Pandey","C Munro","SA Yadav","YK Pathan","AD Russell","PP Chawla","JW Hastings","GB Hogg","UT Yadav"],["NM Coulter-Nile","Z Khan","NM Coulter-Nile","CH Morris","Z Khan","CH Morris","CR Brathwaite","A Mishra","CR Brathwaite","A Mishra","CH Morris","NM Coulter-Nile","CH Morris","NM Coulter-Nile","Z Khan","A Mishra","CR Brathwaite","Z Khan","CR Brathwaite","A Mishra"]]

match(innings1,innings2)

3 MA Agarwal Q de Kock AD Russell 120
0 Q de Kock MA Agarwal AD Russell 119
1 Q de Kock MA Agarwal AD Russell 118
0 MA Agarwal Q de Kock AD Russell 117
6 MA Agarwal Q de Kock AD Russell 116
W MA Agarwal Q de Kock AD Russell 115
3 Q de Kock SS Iyer UT Yadav 114
6 SS Iyer Q de Kock UT Yadav 113
6 SS Iyer Q de Kock UT Yadav 112
W SS Iyer Q de Kock UT Yadav 111
3 KK Nair Q de Kock UT Yadav 110
3 Q de Kock KK Nair UT Yadav 109
0 Q de Kock KK Nair AD Russell 108
6 Q de Kock KK Nair AD Russell 107
6 Q de Kock KK Nair AD Russell 106
6 Q de Kock KK Nair AD Russell 105
6 Q de Kock KK Nair AD Russell 104
6 Q de Kock KK Nair AD Russell 103
3 KK Nair Q de Kock JW Hastings 102
0 Q de Kock KK Nair JW Hastings 101
0 Q de Kock KK Nair JW Hastings 100
0 Q de Kock KK Nair JW Hastings 99
4 Q de Kock KK Nair JW Hastings 98
0 Q de Kock KK Nair JW Hastings 97
0 KK Nair Q de Kock AD Russell 96
2 KK Nair Q de Kock AD Russell 95
W KK Nair Q de Kock AD Russell 94
4 SV Samson Q de Kock AD Russell 93
1 SV Samson Q

In [61]:
#match 2
#3rd match ipl 2016
innings1 = [["M Vijay","M Vohra","DA Miller","GJ Maxwell","WP Saha","MP Stoinis","AR Patel","MG Johnson","P Sahu","MM Sharma","Sandeep Sharma"],["P Kumar","PJ Sangwan","P Kumar","PJ Sangwan","JP Faulkner","S Ladda","RA Jadeja","S Ladda","RA Jadeja","DJ Bravo","RA Jadeja","DJ Bravo","RA Jadeja","DJ Bravo","JP Faulkner","P Kumar","JP Faulkner","P Kumar","JP Faulkner","DJ Bravo"]]

innings2 = [["AJ Finch","BB McCullum","SK Raina","KD Karthik","RA Jadeja","Ishan Kishan","DJ Bravo","JP Faulkner","P Kumar","PJ Sangwan","S Ladda"],["Sandeep Sharma","MG Johnson","Sandeep Sharma","MM Sharma","MG Johnson","MP Stoinis","AR Patel","MP Stoinis","AR Patel","P Sahu","Sandeep Sharma","P Sahu","MM Sharma","P Sahu","MG Johnson","P Sahu","MG Johnson","MM Sharma","Sandeep Sharma","AR Patel"]]

match(innings1,innings2)


3 M Vijay M Vohra P Kumar 120
2 M Vohra M Vijay P Kumar 119
W M Vohra M Vijay P Kumar 118
1 DA Miller M Vijay P Kumar 117
3 M Vijay DA Miller P Kumar 116
0 DA Miller M Vijay P Kumar 115
3 M Vijay DA Miller PJ Sangwan 114
0 DA Miller M Vijay PJ Sangwan 113
0 DA Miller M Vijay PJ Sangwan 112
1 DA Miller M Vijay PJ Sangwan 111
3 M Vijay DA Miller PJ Sangwan 110
0 DA Miller M Vijay PJ Sangwan 109
0 M Vijay DA Miller P Kumar 108
0 M Vijay DA Miller P Kumar 107
1 M Vijay DA Miller P Kumar 106
1 DA Miller M Vijay P Kumar 105
0 M Vijay DA Miller P Kumar 104
1 M Vijay DA Miller P Kumar 103
1 M Vijay DA Miller PJ Sangwan 102
1 DA Miller M Vijay PJ Sangwan 101
1 M Vijay DA Miller PJ Sangwan 100
1 DA Miller M Vijay PJ Sangwan 99
1 M Vijay DA Miller PJ Sangwan 98
1 DA Miller M Vijay PJ Sangwan 97
1 DA Miller M Vijay JP Faulkner 96
2 M Vijay DA Miller JP Faulkner 95
1 M Vijay DA Miller JP Faulkner 94
1 DA Miller M Vijay JP Faulkner 93
4 M Vijay DA Miller JP Faulkner 92
2 M Vijay DA Miller JP Faulkne

In [105]:
#match 3
#4th match ipl 2016

innings1 = [["CH Gayle","V Kohli","AB de Villiers","SR Watson","SN Khan","KM Jadhav","STR Binny","Parvez Rasool","HV Patel","AF Milne","YS Chahal"],["A Nehra","B Kumar","Mustafizur Rahman","A Nehra","B Kumar","Mustafizur Rahman","MC Henriques","KV Sharma","MC Henriques","KV Sharma","A Ashish Reddy","MC Henriques","KV Sharma","MC Henriques","A Ashish Reddy","B Kumar","KV Sharma","Mustafizur Rahman","B Kumar","Mustafizur Rahman"]]

innings2 = [["DA Warner","S Dhawan","MC Henriques","NV Ojha","DJ Hooda","EJG Morgan","A Ashish Reddy","KV Sharma","A Nehra","B Kumar","Mustafizur Rahman"],["AF Milne","SR Watson","AF Milne","Parvez Rasool","HV Patel","Parvez Rasool","HV Patel","YS Chahal","SR Watson","YS Chahal","AF Milne","YS Chahal","Parvez Rasool","YS Chahal","Parvez Rasool","SR Watson","HV Patel","AF Milne","HV Patel","SR Watson"]]


match(innings1,innings2)



3 CH Gayle V Kohli A Nehra 120
0 V Kohli CH Gayle A Nehra 119
1 V Kohli CH Gayle A Nehra 118
3 CH Gayle V Kohli A Nehra 117
3 V Kohli CH Gayle A Nehra 116
4 CH Gayle V Kohli A Nehra 115
W V Kohli CH Gayle B Kumar 114
4 AB de Villiers CH Gayle B Kumar 113
2 AB de Villiers CH Gayle B Kumar 112
1 AB de Villiers CH Gayle B Kumar 111
0 CH Gayle AB de Villiers B Kumar 110
0 CH Gayle AB de Villiers B Kumar 109
4 AB de Villiers CH Gayle Mustafizur Rahman 108
1 AB de Villiers CH Gayle Mustafizur Rahman 107
0 CH Gayle AB de Villiers Mustafizur Rahman 106
1 CH Gayle AB de Villiers Mustafizur Rahman 105
6 AB de Villiers CH Gayle Mustafizur Rahman 104
6 AB de Villiers CH Gayle Mustafizur Rahman 103
4 CH Gayle AB de Villiers A Nehra 102
1 CH Gayle AB de Villiers A Nehra 101
4 AB de Villiers CH Gayle A Nehra 100
0 AB de Villiers CH Gayle A Nehra 99
3 AB de Villiers CH Gayle A Nehra 98
0 CH Gayle AB de Villiers A Nehra 97
0 AB de Villiers CH Gayle B Kumar 96
0 AB de Villiers CH Gayle B Kumar 95
0 AB d

In [62]:
#match 4
#55th match ipl 2016
innings1 = [["RV Uthappa","G Gambhir","C Munro","MK Pandey","YK Pathan","JO Holder","Shakib Al Hasan","SA Yadav","AS Rajpoot", "SP Narine", "Kuldeep Yadav"],["B Kumar","BB Sran","B Kumar","BB Sran","KS Williamson","DJ Hooda","KV Sharma","DJ Hooda","KV Sharma","MC Henriques","Mustafizur Rahman","MC Henriques","Mustafizur Rahman","KV Sharma","BB Sran","B Kumar","Mustafizur Rahman","BB Sran","Mustafizur Rahman","B Kumar"]]

innings2 = [['S Dhawan','DA Warner ','NV Ojha ','Yuvraj Singh','KS Williamson','DJ Hooda','MC Henriques','KV Sharma', 'B Kumar','BB Sran', 'Mustafizur Rahman'],["YK Pathan","AS Rajpoot","Shakib Al Hasan","SP Narine","AS Rajpoot","SP Narine","JO Holder","C Munro","JO Holder","Kuldeep Yadav","Shakib Al Hasan","Kuldeep Yadav","SP Narine","Kuldeep Yadav","Shakib Al Hasan","Kuldeep Yadav","Shakib Al Hasan","SP Narine","JO Holder","AS Rajpoot"]]

match(innings1,innings2)



0 RV Uthappa G Gambhir B Kumar 120
0 RV Uthappa G Gambhir B Kumar 119
0 RV Uthappa G Gambhir B Kumar 118
1 RV Uthappa G Gambhir B Kumar 117
1 G Gambhir RV Uthappa B Kumar 116
1 RV Uthappa G Gambhir B Kumar 115
0 RV Uthappa G Gambhir BB Sran 114
6 RV Uthappa G Gambhir BB Sran 113
6 RV Uthappa G Gambhir BB Sran 112
6 RV Uthappa G Gambhir BB Sran 111
0 RV Uthappa G Gambhir BB Sran 110
6 RV Uthappa G Gambhir BB Sran 109
W G Gambhir RV Uthappa B Kumar 108
W C Munro RV Uthappa B Kumar 107
1 MK Pandey RV Uthappa B Kumar 106
0 RV Uthappa MK Pandey B Kumar 105
1 RV Uthappa MK Pandey B Kumar 104
3 MK Pandey RV Uthappa B Kumar 103
1 MK Pandey RV Uthappa BB Sran 102
3 RV Uthappa MK Pandey BB Sran 101
3 MK Pandey RV Uthappa BB Sran 100
1 RV Uthappa MK Pandey BB Sran 99
W MK Pandey RV Uthappa BB Sran 98
W YK Pathan RV Uthappa BB Sran 97
0 RV Uthappa JO Holder KS Williamson 96
0 RV Uthappa JO Holder KS Williamson 95
2 RV Uthappa JO Holder KS Williamson 94
0 RV Uthappa JO Holder KS Williamson 93
3 RV 

In [63]:
#match 5

#56th match ipl 2016
innings1 = [['Q de Kock ', 'RR Pant', 'KK Nair', 'SV Samson', 'SW Billings', 'P Negi', 'CR Brathwaite', 'CH Morris', 'J Yadav', 'A Mishra', 'Z Khan'],['STR Binny', 'S Aravind', 'CJ Jordan', 'S Aravind', 'SR Watson', 'YS Chahal', 'STR Binny', 'YS Chahal', 'SR Watson', 'YS Chahal', 'Iqbal Abdulla', 'CJ Jordan', 'CH Gayle', 'S Aravind', 'CH Gayle', 'S Aravind', 'YS Chahal', 'SR Watson', 'Iqbal Abdulla', 'SR Watson']]

innings2=[['CH Gayle', 'V Kohli ', 'AB de Villiers', 'KL Rahul ', 'SR Watson', 'STR Binny', 'Sachin Baby ', 'CJ Jordan', 'Iqbal Abdulla', 'S Aravind', 'YS Chahal'],['Z Khan', 'CH Morris', 'Z Khan', 'CH Morris', 'Z Khan', 'CH Morris', 'P Negi', 'A Mishra', 'P Negi', 'J Yadav', 'CR Brathwaite', 'A Mishra', 'CR Brathwaite', 'A Mishra', 'P Negi', 'A Mishra', 'CR Brathwaite', 'Z Khan', 'CR Brathwaite', 'CH Morris']]


match(innings1,innings2)



1 Q de Kock  RR Pant STR Binny 120
1 RR Pant Q de Kock  STR Binny 119
4 Q de Kock  RR Pant STR Binny 118
W Q de Kock  RR Pant STR Binny 117
2 KK Nair RR Pant STR Binny 116
4 KK Nair RR Pant STR Binny 115
6 RR Pant KK Nair S Aravind 114
6 RR Pant KK Nair S Aravind 113
6 RR Pant KK Nair S Aravind 112
6 RR Pant KK Nair S Aravind 111
6 RR Pant KK Nair S Aravind 110
6 RR Pant KK Nair S Aravind 109
4 KK Nair RR Pant CJ Jordan 108
2 KK Nair RR Pant CJ Jordan 107
3 KK Nair RR Pant CJ Jordan 106
W RR Pant KK Nair CJ Jordan 105
0 SV Samson KK Nair CJ Jordan 104
1 SV Samson KK Nair CJ Jordan 103
3 SV Samson KK Nair S Aravind 102
3 KK Nair SV Samson S Aravind 101
3 SV Samson KK Nair S Aravind 100
1 KK Nair SV Samson S Aravind 99
1 SV Samson KK Nair S Aravind 98
4 KK Nair SV Samson S Aravind 97
W SV Samson KK Nair SR Watson 96
0 SW Billings KK Nair SR Watson 95
W SW Billings KK Nair SR Watson 94
3 P Negi KK Nair SR Watson 93
1 KK Nair P Negi SR Watson 92
0 P Negi KK Nair SR Watson 91
W KK Nair P Ne

In [12]:
#match 6

#Match 57 of ipl 2016

innings1 = [['AJ Finch', 'BB McCullum', 'SK Raina ', 'KD Karthik', 'DR Smith', 'RA Jadeja', 'DJ Bravo', 'ER Dwivedi', 'P Kumar', 'DS Kulkarni', 'SB Jakati'],['S Aravind', 'Iqbal Abdulla', 'S Aravind', 'SR Watson', 'Iqbal Abdulla', 'SR Watson', 'CJ Jordan', 'YS Chahal', 'STR Binny', 'YS Chahal', 'Iqbal Abdulla', 'YS Chahal', 'Iqbal Abdulla', 'CJ Jordan', 'S Aravind', 'SR Watson', 'YS Chahal', 'CJ Jordan', 'SR Watson', 'CJ Jordan']]

innings2 = [['CH Gayle', 'V Kohli', 'AB de Villiers', 'KL Rahul', 'SR Watson', 'Sachin Baby', 'STR Binny', 'Iqbal Abdulla', 'CJ Jordan', 'S Aravind', 'YS Chahal'],['P Kumar', 'DS Kulkarni', 'P Kumar', 'DS Kulkarni', 'RA Jadeja', 'DS Kulkarni', 'RA Jadeja', 'DS Kulkarni', 'SB Jakati', 'RA Jadeja', 'SB Jakati', 'DJ Bravo', 'RA Jadeja', 'DJ Bravo', 'DR Smith', 'SB Jakati', 'P Kumar', 'DJ Bravo', 'P Kumar', 'DJ Bravo']]


match(innings1,innings2)


0 AJ Finch BB McCullum S Aravind 120
4 AJ Finch BB McCullum S Aravind 119
0 AJ Finch BB McCullum S Aravind 118
W AJ Finch BB McCullum S Aravind 117
W SK Raina  BB McCullum S Aravind 116
0 KD Karthik BB McCullum S Aravind 115
1 BB McCullum KD Karthik Iqbal Abdulla 114
3 KD Karthik BB McCullum Iqbal Abdulla 113
2 BB McCullum KD Karthik Iqbal Abdulla 112
1 BB McCullum KD Karthik Iqbal Abdulla 111
2 KD Karthik BB McCullum Iqbal Abdulla 110
1 KD Karthik BB McCullum Iqbal Abdulla 109
0 KD Karthik BB McCullum S Aravind 108
W KD Karthik BB McCullum S Aravind 107
4 DR Smith BB McCullum S Aravind 106
6 DR Smith BB McCullum S Aravind 105
3 DR Smith BB McCullum S Aravind 104
2 BB McCullum DR Smith S Aravind 103
0 DR Smith BB McCullum SR Watson 102
1 DR Smith BB McCullum SR Watson 101
4 BB McCullum DR Smith SR Watson 100
4 BB McCullum DR Smith SR Watson 99
3 BB McCullum DR Smith SR Watson 98
4 DR Smith BB McCullum SR Watson 97
6 BB McCullum DR Smith Iqbal Abdulla 96
6 BB McCullum DR Smith Iqbal Abd

In [64]:
#match 7

#match 59 of ipl 2016

innings1 =[['ER Dwivedi', 'BB McCullum', 'SK Raina', 'KD Karthik', 'AJ Finch', 'DR Smith', 'RA Jadeja', 'DJ Bravo', 'DS Kulkarni', 'P Kumar', 'S Kaushik'],['B Kumar', 'TA Boult', 'B Kumar', 'TA Boult', 'BB Sran', 'Bipul Sharma', 'BCJ Cutting', 'MC Henriques', 'Bipul Sharma', 'MC Henriques', 'BB Sran', 'Bipul Sharma', 'BCJ Cutting', 'MC Henriques', 'BB Sran', 'TA Boult', 'B Kumar', 'BCJ Cutting', 'TA Boult', 'B Kumar']]

innings2 = [['DA Warner', 'S Dhawan', 'MC Henriques', 'Yuvraj Singh', 'DJ Hooda', 'BCJ Cutting', 'NV Ojha', 'Bipul Sharma', 'B Kumar', 'BB Sran', 'TA Boult'],['P Kumar', 'DS Kulkarni', 'P Kumar', 'DS Kulkarni', 'DR Smith', 'SK Raina', 'S Kaushik','SK Raina', 'S Kaushik', 'DJ Bravo', 'S Kaushik', 'DJ Bravo', 'S Kaushik', 'DS Kulkarni', 'DR Smith', 'DJ Bravo', 'P Kumar', 'DS Kulkarni', 'DJ Bravo', 'P Kumar']]


match(innings1,innings2)



3 ER Dwivedi BB McCullum B Kumar 120
1 BB McCullum ER Dwivedi B Kumar 119
0 ER Dwivedi BB McCullum B Kumar 118
0 ER Dwivedi BB McCullum B Kumar 117
0 ER Dwivedi BB McCullum B Kumar 116
W ER Dwivedi BB McCullum B Kumar 115
1 BB McCullum SK Raina TA Boult 114
6 SK Raina BB McCullum TA Boult 113
W SK Raina BB McCullum TA Boult 112
1 KD Karthik BB McCullum TA Boult 111
2 BB McCullum KD Karthik TA Boult 110
2 BB McCullum KD Karthik TA Boult 109
1 KD Karthik BB McCullum B Kumar 108
0 BB McCullum KD Karthik B Kumar 107
W BB McCullum KD Karthik B Kumar 106
2 AJ Finch KD Karthik B Kumar 105
W AJ Finch KD Karthik B Kumar 104
0 DR Smith KD Karthik B Kumar 103
W KD Karthik DR Smith TA Boult 102
4 RA Jadeja DR Smith TA Boult 101
3 RA Jadeja DR Smith TA Boult 100
4 DR Smith RA Jadeja TA Boult 99
4 DR Smith RA Jadeja TA Boult 98
1 DR Smith RA Jadeja TA Boult 97
1 DR Smith RA Jadeja BB Sran 96
3 RA Jadeja DR Smith BB Sran 95
0 DR Smith RA Jadeja BB Sran 94
1 DR Smith RA Jadeja BB Sran 93
1 RA Jadeja D

In [69]:
#match 8

#match 60 of ipl 2016

innings1 = [['DA Warner', 'S Dhawan', 'MC Henriques', 'Yuvraj Singh', 'DJ Hooda', 'BCJ Cutting', 'NV Ojha', 'Bipul Sharma', 'B Kumar', 'BB Sran', 'Mustafizur Rahman'],['S Aravind', 'CH Gayle', 'S Aravind', 'CH Gayle', 'SR Watson', 'CH Gayle', 'YS Chahal', 'Iqbal Abdulla', 'YS Chahal', 'CJ Jordan', 'SR Watson', 'CJ Jordan', 'YS Chahal', 'S Aravind', 'YS Chahal', 'S Aravind', 'CJ Jordan', 'SR Watson', 'CJ Jordan', 'SR Watson']]

innings2 =[['CH Gayle', 'V Kohli', 'AB de Villiers', 'KL Rahul', 'SR Watson', 'Sachin Baby', 'STR Binny', 'CJ Jordan', 'Iqbal Abdulla', 'S Arvind', 'YS Chahal'],['B Kumar', 'BB Sran', 'B Kumar', 'BB Sran', 'BCJ Cutting', 'Mustafizur Rahman', 'MC Henriques', 'BCJ Cutting', 'MC Henriques', 'Mustafizur Rahman', 'BCJ Cutting', 'Bipul Sharma', 'BB Sran', 'Bipul Sharma', 'MC Henriques', 'BCJ Cutting', 'Mustafizur Rahman', 'B Kumar', 'Mustafizur Rahman', 'B Kumar']]

match(innings1,innings2)



4 DA Warner S Dhawan S Aravind 120
2 DA Warner S Dhawan S Aravind 119
2 DA Warner S Dhawan S Aravind 118
3 DA Warner S Dhawan S Aravind 117
0 S Dhawan DA Warner S Aravind 116
1 S Dhawan DA Warner S Aravind 115
0 S Dhawan DA Warner CH Gayle 114
3 S Dhawan DA Warner CH Gayle 113
3 DA Warner S Dhawan CH Gayle 112
0 S Dhawan DA Warner CH Gayle 111
0 S Dhawan DA Warner CH Gayle 110
2 S Dhawan DA Warner CH Gayle 109
1 DA Warner S Dhawan S Aravind 108
2 S Dhawan DA Warner S Aravind 107
0 S Dhawan DA Warner S Aravind 106
3 S Dhawan DA Warner S Aravind 105
1 DA Warner S Dhawan S Aravind 104
0 S Dhawan DA Warner S Aravind 103
1 DA Warner S Dhawan CH Gayle 102
3 S Dhawan DA Warner CH Gayle 101
3 DA Warner S Dhawan CH Gayle 100
0 S Dhawan DA Warner CH Gayle 99
W S Dhawan DA Warner CH Gayle 98
W MC Henriques DA Warner CH Gayle 97
3 DA Warner Yuvraj Singh SR Watson 96
0 Yuvraj Singh DA Warner SR Watson 95
W Yuvraj Singh DA Warner SR Watson 94
0 DJ Hooda DA Warner SR Watson 93
3 DJ Hooda DA Warner SR

In [66]:
#match 9

#match 53 of ipl 2016

innings1 = [["HM AMla","M Vijay","WP Saha","Gurkeerat Singh","DA Miller","F Behardien","AR Patel","R Dhawan","KJ Abbott","MM Sharma","Sandeep Sharma"],["IK Pathan","AB Dinda","DL Chahar","AB Dinda","DL Chahar","NLTC Perera","IK Pathan","R Aswhin","A Zampa","R Aswhin","A Zampa","DL Chahar","A Zampa","IK Pathan","A Zampa","R Aswhin","IK Pathan","R Aswhin","AB Dinda","NLTC Perera"]]


innings2 = [["AM Rahane","UT Khawaja","GJ Bailey","SS Tiwary","MS Dhoni","IK Pathan","NLTC Perera","R Ashwin","A Zampa","AB Dinda","DL Chahar"],["Sandeep Sharma","MM Sharma","Sandeep Sharma","KJ Abbott","Sandeep Sharma","MM Sharma","AR Patel","Gurkeerat Singh","R Dhawan","KJ Abbott","R Dhawan","Gurkeerat Singh","AR Patel","R Dhawan","AR Patel","KJ Abbott","MM Sharma","Sandeep Sharma","MM Sharma","AR Patel"]]

match(innings1,innings2)

W HM AMla M Vijay IK Pathan 120
3 WP Saha M Vijay IK Pathan 119
1 M Vijay WP Saha IK Pathan 118
W WP Saha M Vijay IK Pathan 117
W Gurkeerat Singh M Vijay IK Pathan 116
3 DA Miller M Vijay IK Pathan 115
1 DA Miller M Vijay AB Dinda 114
4 M Vijay DA Miller AB Dinda 113
0 M Vijay DA Miller AB Dinda 112
1 M Vijay DA Miller AB Dinda 111
W DA Miller M Vijay AB Dinda 110
W F Behardien M Vijay AB Dinda 109
2 M Vijay AR Patel DL Chahar 108
4 M Vijay AR Patel DL Chahar 107
3 M Vijay AR Patel DL Chahar 106
3 AR Patel M Vijay DL Chahar 105
2 M Vijay AR Patel DL Chahar 104
3 M Vijay AR Patel DL Chahar 103
1 M Vijay AR Patel AB Dinda 102
2 AR Patel M Vijay AB Dinda 101
3 AR Patel M Vijay AB Dinda 100
1 M Vijay AR Patel AB Dinda 99
4 AR Patel M Vijay AB Dinda 98
2 AR Patel M Vijay AB Dinda 97
3 M Vijay AR Patel DL Chahar 96
4 AR Patel M Vijay DL Chahar 95
1 AR Patel M Vijay DL Chahar 94
3 M Vijay AR Patel DL Chahar 93
1 AR Patel M Vijay DL Chahar 92
2 M Vijay AR Patel DL Chahar 91
6 AR Patel M Vijay 

In [18]:
#match 10

#match 54 of ipl 2016
innings1 = [["RG Sharma", "MJ Guptill","N Rana","KH Pandya","JC Buttler","KA Pollard","HH Pandya","Harbhajan Singh","R Vinay Kumar","MJ McClenaghan","JJ Bumrah"],["P Kumar","DS Kulkarni","P Kumar","DS Kulkarni","DR Smith","P Kumar","DR Smith","SB Jakati","DR Smith","DJ Bravo","RA Jadeja","DR Smith","SB Jakati","DJ Bravo","SB Jakati","DS Kulkarni","DJ Bravo","DS Kulkarni","DJ Bravo","P Kumar"]]

innings2 = [["AJ Finch","BB McCullum","SK Raina","KD Karthik","DR Smith","RA Jadeja","DJ Bravo","ER Dwivedi","SB Jakati","P Kumar","DS Kulkarni"],["R Vinay Kumar","JJ Bumrah","MJ McClenaghan","R Vinay Kumar","KH Pandya","JJ Bumrah","KH Pandya","Harbhajan Singh","MJ McClenaghan","Harbhajan Singh","R Vinay Kumar","Harbhajan Singh","JJ Bumrah","Harbhajan Singh","MJ McClenaghan","HH Pandya","JJ Bumrah","MJ McClenaghan","R Vinay Kumar","HH Pandya"]]

match(innings1,innings2)


0 RG Sharma MJ Guptill P Kumar 120
3 RG Sharma MJ Guptill P Kumar 119
1 MJ Guptill RG Sharma P Kumar 118
0 RG Sharma MJ Guptill P Kumar 117
4 RG Sharma MJ Guptill P Kumar 116
0 RG Sharma MJ Guptill P Kumar 115
3 MJ Guptill RG Sharma DS Kulkarni 114
0 RG Sharma MJ Guptill DS Kulkarni 113
4 RG Sharma MJ Guptill DS Kulkarni 112
3 RG Sharma MJ Guptill DS Kulkarni 111
0 MJ Guptill RG Sharma DS Kulkarni 110
4 MJ Guptill RG Sharma DS Kulkarni 109
0 RG Sharma MJ Guptill P Kumar 108
3 RG Sharma MJ Guptill P Kumar 107
0 MJ Guptill RG Sharma P Kumar 106
0 MJ Guptill RG Sharma P Kumar 105
0 MJ Guptill RG Sharma P Kumar 104
2 MJ Guptill RG Sharma P Kumar 103
3 RG Sharma MJ Guptill DS Kulkarni 102
0 MJ Guptill RG Sharma DS Kulkarni 101
3 MJ Guptill RG Sharma DS Kulkarni 100
0 RG Sharma MJ Guptill DS Kulkarni 99
2 RG Sharma MJ Guptill DS Kulkarni 98
0 RG Sharma MJ Guptill DS Kulkarni 97
3 MJ Guptill RG Sharma DR Smith 96
0 RG Sharma MJ Guptill DR Smith 95
1 RG Sharma MJ Guptill DR Smith 94
6 MJ Gupti

1 BB McCullum RA Jadeja HH Pandya 30
3 RA Jadeja BB McCullum HH Pandya 29
0 BB McCullum RA Jadeja HH Pandya 28
0 BB McCullum RA Jadeja HH Pandya 27
1 BB McCullum RA Jadeja HH Pandya 26
3 RA Jadeja BB McCullum HH Pandya 25
0 RA Jadeja BB McCullum JJ Bumrah 24
0 RA Jadeja BB McCullum JJ Bumrah 23
1 RA Jadeja BB McCullum JJ Bumrah 22
6 BB McCullum RA Jadeja JJ Bumrah 21
6 BB McCullum RA Jadeja JJ Bumrah 20
Chasing Team wins

For batsman : [runs,dots,4s,6s,balls,strike rate , out_by]

{'AJ Finch': [40, 5, 1, 4, 17, 235.29411764705884, 'KH Pandya'], 'BB McCullum': [118, 17, 11, 3, 56, 210.71428571428572, ''], 'SK Raina': [2, 2, 0, 0, 3, 66.66666666666666, 'KH Pandya'], 'KD Karthik': [6, 1, 0, 0, 3, 200.0, 'MJ McClenaghan'], 'DR Smith': [20, 8, 2, 0, 17, 117.64705882352942, 'Harbhajan Singh'], 'RA Jadeja': [7, 2, 0, 0, 5, 140.0, ''], 'DJ Bravo': [0, 0, 0, 0, 0, 0, ''], 'ER Dwivedi': [0, 0, 0, 0, 0, 0, ''], 'SB Jakati': [0, 0, 0, 0, 0, 0, ''], 'P Kumar': [0, 0, 0, 0, 0, 0, ''], 'DS Kulkarni':

In [75]:
l = [1,1,1,0,0,1,0,1,0,1]
acc = sum(l)/10* 100
print(acc)

60.0


In [22]:
innings1 = [["PP Shaw","C Munro","SS Iyer","RR Pant","GJ Maxwell","V Shankar","LE Plunkett","A Mishra","S Nadeem","Avesh Khan","TA Boult"],["DS Kulkarni","JC Archer","DS Kulkarni","K Gowtham","JD Unadkat","S Gopal","BA Stokes","S Gopal","BA Stokes","JC Archer","JD Unadkat","K Gowtham","DS Kulkarni","BA Stokes","JD Unadkat","JC Archer","JD Unadkat","JC Archer","DS Kulkarni","BA Stokes"]]
innings2 = [["DJM Short","JC Buttler","SV Samson","BA Stokes","RA Tripathi","K Gowtham","JC Archer","AM Rahane","JD Unadkat","DS Kulkarni","S Gopal"],["S Nadeem","TA Boult","Avesh Khan","L Plunkett","A Mishra","Avesh Khan","A Mishra","L Plunkett","TA Boult","GJ Maxwell","TA Boult","A Mishra","S Nadeem","A Mishra","S Nadeem","TA Boult","L Plunkett","Avesh Khan","S Nadeem","Avesh Khan"]]

match(innings1,innings2)


1 PP Shaw C Munro DS Kulkarni 120
0 C Munro PP Shaw DS Kulkarni 119
6 C Munro PP Shaw DS Kulkarni 118
0 C Munro PP Shaw DS Kulkarni 117
4 C Munro PP Shaw DS Kulkarni 116
1 C Munro PP Shaw DS Kulkarni 115
4 C Munro PP Shaw JC Archer 114
1 C Munro PP Shaw JC Archer 113
4 PP Shaw C Munro JC Archer 112
4 PP Shaw C Munro JC Archer 111
1 PP Shaw C Munro JC Archer 110
1 C Munro PP Shaw JC Archer 109
1 C Munro PP Shaw DS Kulkarni 108
1 PP Shaw C Munro DS Kulkarni 107
0 C Munro PP Shaw DS Kulkarni 106
0 C Munro PP Shaw DS Kulkarni 105
0 C Munro PP Shaw DS Kulkarni 104
1 C Munro PP Shaw DS Kulkarni 103
1 C Munro PP Shaw K Gowtham 102
1 PP Shaw C Munro K Gowtham 101
0 C Munro PP Shaw K Gowtham 100
0 C Munro PP Shaw K Gowtham 99
1 C Munro PP Shaw K Gowtham 98
0 PP Shaw C Munro K Gowtham 97
1 C Munro PP Shaw JD Unadkat 96
1 PP Shaw C Munro JD Unadkat 95
1 C Munro PP Shaw JD Unadkat 94
0 PP Shaw C Munro JD Unadkat 93
1 PP Shaw C Munro JD Unadkat 92
1 C Munro PP Shaw JD Unadkat 91
0 C Munro PP Shaw S